# Visualizando los datos de los reportes diarios
> Como podemos hacer para aprovechar los datos de los informes diarios

- toc: false 
- badges: true
- comments: false
- categories: [graficos]
- image: images/evolucion-por-provincia.png

En este post evaluamos rapidamente como se puede aprovechar la informacion que se encuentra disponible en el sitio https://www.argentina.gob.ar/coronavirus/informe-diario para tener una vision general de lo que pasa en el pais.

In [1]:
#hide

import arg_data
import pandas as pd

In [2]:
#hide

df = arg_data.get_arg_df()
cnt_by_place = df.groupby('infered_place').infected.sum()

In [3]:
#hide
import altair as alt

def plot_evolution(selected_places=None, xaxis='date', xaxis_title='Fecha'):
    selection = alt.selection_multi(
        fields=['infered_place'], bind='legend', init=[{'infered_place': c} for c in (selected_places or [])]
    )
    
    return (
        alt.Chart(df)
           .mark_line(point=True, radius=150)
           .encode(
               x=alt.X(xaxis, axis=alt.Axis(title=xaxis_title)),
               y=alt.Y('cum_infected', scale=alt.Scale(type='log'), axis=alt.Axis(title='Casos confirmados')),
               color=alt.Color('infered_place', sort=list(cnt_by_place.index)),
               tooltip=list(df.columns), 
               opacity=alt.condition(selection, alt.value(.8), alt.value(.05))
            ).add_selection(selection)
             .configure_point(size=200)
             .properties(width=650, height=400)
             .configure_title(align='center', )
    )
    

# Evolucion en el tiempo de la cantidad de casos totales

Haciendo Shift+Click en las provincias que se ven a la derecha, podemos agregar o quitar provincias al gráfico

In [4]:
#hide_input
plot_evolution(cnt_by_place[cnt_by_place >=10].index.tolist())#, xaxis='days_from_first_infection')

alt.Chart(...)

# Graficando las mismas curvas cambiando el eje X

En lugar de utilizar la fecha, utilizamos cantidad de dias desde el primer caso confirmado

In [5]:
#hide_input
plot_evolution(
    cnt_by_place[cnt_by_place >=10].index.tolist(), 
    xaxis='days_from_first_infection', 
    xaxis_title='Dias desde el primer caso confirmado'
)

alt.Chart(...)

Los datos de este código ignoran los primeros informes, correspondiente a los primeros días de la infección. 

Los primeros informes, con los primeros casos eran mucho mas desestructurados, y es más dificultoso extraer la información. 

En los nuevos se itemiza por provincia, y como representan la gran mayoria de los casos, no están en estas estadísticas. 